In [29]:
import os
import torch
import requests
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

from langchain import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint



In [30]:
def config():
    load_dotenv()

config()

"top_k":1,
                     "top_p":0.9,
                     "temperature":0.98,
                     "max_length":500

In [92]:
repo_id = "google/gemma-7b-it"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    top_k=2,
    top_p=0.4,    
    temperature=0.98,
    max_new_tokens = 500,
    huggingfacehub_api_token=os.getenv('API_KEY')
)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\Mayur\.cache\huggingface\token
Login successful


In [93]:
# To check input given to model.
def gemma_template(prompt):
    text =  "<start_of_turn>user\n{}<end_of_turn>".format(prompt) + "\n<start_of_turn>model"    
    return text

In [94]:
question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: """

prompt = gemma_template(template)
prompt = PromptTemplate.from_template(prompt)
prompt


PromptTemplate(input_variables=['question'], template='<start_of_turn>user\nQuestion: {question}\n\nAnswer: <end_of_turn>\n<start_of_turn>model')

In [95]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [96]:
print(llm_chain.run(question))

**Answer:** The answer is Brazil. Brazil won the FIFA World Cup in 1994.
